In [ ]:
import keras
import tensorflow as tf
from keras.applications import vgg16
from keras.models import *
from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import warnings
import math

In [ ]:
aux_input = Input(shape=(512, 512, 3))
base = vgg16.VGG16(include_top=False, weights='./vgg16_weights.h5', input_tensor=aux_input)
cnn = base.output

In [ ]:
for layer in base.layers:
    layer.trainable = False

In [ ]:
def myactivation(x,minimum=0,maximum=512):
    x = tf.sigmoid(x)+1
    scale = (maximum-minimum)/2
    return x*scale+minimum

In [ ]:
ffx = Dense(200,activation='relu')(cnn)
ffx = Dropout(.5)(ffx)
flat = Flatten()(ffx)
ffx = Dense(250,activation='relu')(flat)
ffx = Dropout(.5)(ffx)
ffx = RepeatVector(80)(ffx)
lstm = LSTM(100,activation='tanh',return_sequences=True,unroll=True,dropout=.5)(ffx)
ffy = Dense(50,activation='relu')(lstm)
ffy = Dropout(.5)(ffy)
out1 = Dense(80,activation='softmax',name='out1')(ffy)

In [ ]:
s0 = Dense(1,activation='relu')(cnn)
s0 = Flatten()(s0)
out2 = Dense(256,activation='softmax',name='out2')(s0)

In [ ]:
model = Model(inputs=aux_input,outputs=[out1,out2])
#model2 = Model(inputs=aux_input,outputs=s0)

In [ ]:
model.summary()

In [ ]:
s0_pred = ''
def myloss(y_true, y_pred):
    global s0_pred
    if 40 in y_pred.shape:
        return tf.reduce_mean(tf.square((y_pred - y_true)))
    else:
        print('in myloss else ',y_pred.shape)
        s0_pred = y_pred
        return tf.reduce_mean(tf.square((y_pred - y_true)))

In [ ]:
losses = {'out1':'binary_crossentropy','out2':'binary_crossentropy'}
adam = keras.optimizers.Adam(lr=0.01)
model.compile(optimizer=adam,loss=losses,metrics=['accuracy'])
#model2.compile(optimizer=adam,loss='mse',metrics=['mse'])

In [ ]:
imgpath = 'C:\\Users\\KV Manoj\\pathGAN\\mit eye tracking dataset\\eyetracking data\\ALLSTIMULI\\'
pointspath = 'C:\\Users\\KV Manoj\\pathGAN\\mit eye tracking dataset\\intcsv\\ajs\\'
vimgpath = 'C:\\Users\\KV Manoj\\pathGAN\\mit eye tracking dataset\\eyetracking data\\ALLSTIMULI2\\'
vpointspath = 'C:\\Users\\KV Manoj\\pathGAN\\mit eye tracking dataset\\intcsv2\\CNG2\\'

In [ ]:
def transform(img,i=0):
    img = cv2.resize(img,(512,512))
    if(i==2):
        return img
    img = vgg16.preprocess_input(img)
    if(i!=0):
        img = tf.expand_dims(img,axis=0)
    return img

In [ ]:
x = 'C:\\Users\\KV Manoj\\pathGAN\\mit eye tracking dataset\\intcsv\\'
folders = (os.listdir(x))
x+folders[0]+'\\'

In [ ]:
mod = 0
lis = [[0,2],[2,2]]
for i in range(2,10):
    for j in range(2):
        if(i==9 and j==1):
            break
        x,y = lis[-1]
        if(mod%4==0):
            l=i
            for k in range(y-2,-20,-2):
                if(l>0):
                    lis.append([x,k])
                    l-=1
                else:
                    mod+=1
                    break
        elif(mod%4==1):
            l=i
            for k in range(x-2,-20,-2):
                if(l>0):
                    lis.append([k,y])
                    l-=1
                else:
                    mod+=1
                    break
            
        elif(mod%4==2):
            l=i
            for k in range(y+2,20,2):
                if(l>0):
                    lis.append([x,k])
                    l-=1
                else:
                    mod+=1
                    break
        elif(mod%4==3):
            l=i
            for k in range(x+2,20,2):
                if(l>0):
                    lis.append([k,y])
                    l-=1
                else:
                    mod+=1
                    break
lis.pop()
change = lis

In [ ]:
def data_generator(imgpath,pointspath,lm,bs=16):
    images = sorted(os.listdir(imgpath)) 
    i=0
    j=len(images)
    cd=0
    mod = 0
    pointspath2=pointspath
    while True:
        #print('\n------------------------------------\n')
        imgs = []
        points = []
        first = []
        while len(imgs)<bs:
            #print('len img: ',len(imgs))
            if((i==0 or i==j) and lm==0):
                i=0
                if(pointspath == 'C:\\Users\\KV Manoj\\pathGAN\\mit eye tracking dataset\\intcsv\\ajs\\'):
                    pointspath = 'C:\\Users\\KV Manoj\\pathGAN\\mit eye tracking dataset\\intcsv\\'
                pointspath2 = pointspath + folders[mod%15]+'\\'
                mod+=1
                #print('\nlm=0\n')
                print('\n',pointspath2)
                #print('after pointspath \n------------------------------------\n')
            elif(i==j):
                i=0
                #print('\nlm=1\n')
            if((i)%100==0):
                print(i,end=',')
            #print('hello')
            name = images[i]
            img=''
            if '.jpeg' in name:
                img = plt.imread(imgpath+name)
                img = transform(img)
            else:
                i+=1
                continue
            front = name.split('.jpeg')[0]
            #print('\nfront: ',front)
            csv = front+'.csv'
            df = 0
            try:
                file = pointspath2 + csv
                #print('x: ',x)
                df = pd.read_csv(file)
            except Exception as e:
                print(file,"  -- doesn't exist")
                i+=1
                continue
            df = df[:120][['x','y']]
            z = [[x,y] for x,y in zip(df['x'],df['y'])]
            zz = []
            for m in z[1:]:
                diff = [int(z[0][0]-m[0]),int(z[0][1]-m[1])]
                diff = list(np.clip(diff,-8,8))
                if diff in z:
                    if(diff[0]==0 and diff[1]==0):
                        continue
                    ind = change.index(diff)
                    bits = [0]*80
                    bits[ind] = 1
                    zz.append(bits)
                else:
                    o,p = diff
                    o = int(o)
                    p = int(p)
                    if(o%2!=0 and o>0):
                        o=o-1
                    elif(o%2!=0 and o<0):
                        o=o+1
                    if(p%2!=0 and p>0):
                        p=p-1
                    elif(p%2!=0 and p<0):
                        p=p+1
                    if(o==0 and p==0):
                        continue
                    ind = change.index(list([o,p]))
                    bits = [0]*80
                    bits[ind] = 1
                    zz.append(bits)
            zz = np.array(zz[:80])
            vm = [0]*256
            c = np.clip(np.array(z[0]),0,511)
            px,py = c
            px = px//32
            py = py//32
            vl = py*16+px
            #print(z[0],' : ',vl,px,py)
            if(math.isnan(vl)):
                cd+=1
                i+=1
                continue
            vm[int(vl)%256] = 1
            vm2 = np.array(vm)
            #print('img.shape: ',img.shape,'zz.shape: ',zz.shape)
            if(img.shape==(512,512,3) and zz.shape==(80,80)):
                imgs.append(img)
                points.append(zz)
                first.append(vm2)
            cd+=1
            i+=1
        yield (np.array(imgs),{'out1':np.array(points),'out2':np.array(first)})

In [ ]:
datag=data_generator(imgpath,pointspath,lm=0)
validg = data_generator(vimgpath,vpointspath,lm=1)

In [ ]:
df = pd.read_csv(pointspath+'i05june05_static_street_boston_p1010764.csv')
df.columns

In [ ]:
numofimages = len(os.listdir(imgpath))
numofimages

In [ ]:
model.fit_generator(datag,steps_per_epoch=numofimages//16,epochs=10,validation_data=validg,validation_steps=1)

In [ ]:
model.save('./scanpath_model.h5')

In [ ]:
from keras.models import load_model
model = load_model('./scanpath_model.h5')

In [ ]:
model.summary()

In [ ]:
img = plt.imread('./nature.jpg')
plt.imshow(transform(img,2))
vals=model.predict(transform(img,1),steps=1)

In [ ]:
vals[0]

In [ ]:
ret=beam_search_decoder(vals[0],vals[1])

In [ ]:
ret.shape

In [ ]:
ret[0][1][:,0]

In [ ]:
imgi4 = plt.imread('./i4.jpeg')
df = pd.read_csv('./i4.csv')
x = df['x']
y = df['y']
plt.imshow(imgi4)
plt.plot(x,y)

In [ ]:
imgi4 = plt.imread('./i4.jpeg')
df = pd.read_csv('./i4_2.csv')
x = df['x']
y = df['y']
plt.imshow(imgi4)
plt.plot(x,y)

In [ ]:
xv = plt.imshow(transform(img,2))
xv = plt.plot(np.clip(ret[0][10][:,0],0,512),np.clip(ret[0][10][:,1],0,512))

In [ ]:
xv = plt.imshow(transform(img,2))
xv = plt.plot(np.clip(ret[0][90][:,0],0,512),np.clip(ret[0][90][:,1],0,512))

In [ ]:
xv

In [ ]:
for x in s0_pred:
    print(x)

In [ ]:
numofimages = (os.listdir(pointspath))
numofimages


In [ ]:
def beam_search_decoder(data,first, k=100):
    lis = list()
    for m in range(data.shape[0]):
        sequences = [[list(), 1.0]]
        #candidate = list()
        # walk over each step in sequence
        for row in data[m]:
            all_candidates = list()
            # expand each current candidate
            for i in range(len(sequences)):
                seq, score = sequences[i]
                for j in range(len(row)):
                    candidate = [seq + [j], score * -np.log(row[j])]
                    all_candidates.append(candidate)
            # order all candidates by score
            ordered = sorted(all_candidates, key=lambda tup:tup[1],reverse=True)
            # select k best
            sequences = ordered[:k]
            #print('sequences: ',sequences)
        lis.append(sequences)
    li = list()
    for i in range(len(lis)):
        v = []
        for x in lis[i]:
            v.append(x[0])
        li.append(v)
    #print('lis: ',li)
    ret = decoder(li,first)
    return ret

In [ ]:
def decoder(seqs,first):
    index = np.argmax(first)
    ry = index//16
    rx = index%16
    px = np.random.randint(32*rx+1,32*(rx+1)+1)
    py = np.random.randint(32*ry+1,32*(ry+1)+1)
    points = []
    arr = np.array([[px,py]])
    joints = list()
    for x in arr:
        main = []
        for s in range(len(seqs)):
            for seq in seqs[s]:
                inpoints = []
                inpoints.append(x)
                for y in seq:
                    z = np.array(change[y])
                    inpoints.append(inpoints[-1]+z)
                points.append(inpoints)
        joints.append(points)
    vim = np.array(joints[0])
    #print('joints: ',vim,vim.shape)
        
    return np.array(joints)

In [ ]:
change = [[0,2],[2,2],[2,0],[2,-2],[0,-2]
              ,[-2,-2],[-2,0],[-2,2],[-2,4],[0,4]
              ,[2,4],[4,4],[4,2],[4,0],[4,-2],
              [4,-4],[2,-4],[0,-4],[-2,-4],[-4,-4],
              [-4,-2],[-4,0],[-4,2],[-4,4]]

In [ ]:
rx=0
px = np.random.randint(32*rx+1,32*(rx+1)+1)
px

In [ ]:
l = np.random.randint(-4,4,(24,2))
po = [45,32]
cx = np.array(po) + np.array(l)
print(cx)
zz = cx==np.array(po)

In [ ]:
diff = [4,4]
change.index(diff)

In [ ]:
datag=data_generator(vimgpath,vpointspath)

In [ ]:
len(change)